In [ ]:
"""Imports from Marti Utility scripts"""
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.markers as markers
import scipy.signal as signal
import seaborn as sns
import scipy.stats as stats
from scipy.signal import find_peaks, peak_prominences
from BaselineRemoval import BaselineRemoval
import os

In [ ]:
def rolling_min(input_series, window_size):
    r = input_series.rolling(window_size, min_periods=1)
    m = r.min()
    return m


def remove_bleaching(input_trace):
    min_trace = rolling_min(pd.Series(input_trace), window_size=int(len(input_trace)/10))
    fit_coefficients = np.polyfit(range(len(min_trace)), min_trace, 2)
    fit = np.poly1d(fit_coefficients)
    return input_trace - fit(range(len(input_trace)))

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.markers as markers
import scipy.signal as signal
import seaborn as sns
import scipy.stats as stats
from scipy.signal import find_peaks, peak_prominences
from BaselineRemoval import BaselineRemoval
import os
def load(fp):
    stat = np.load(f"{fp}\stat.npy", allow_pickle = True)
    ops = np.load(f"{fp}\ops.npy", allow_pickle = True).item()
    F = np.load(f"{fp}\F.npy", allow_pickle = True)
    Fneu = np.load(f"{fp}\Fneu.npy", allow_pickle = True)
    #iscell / spks also can be repeated in this way
    return stat, ops, F, Fneu
    
stat, ops, F, Fneu = load(r'E:\001 - Synapse Calcium Imaging\230727_AMPA_blocker_tests_AAV1_syn_GCaMP6\cs2\wtRt_225k_.0.5ul_hSyn_DIV21_100%_25ms_r001_cs1\suite2p\plane0')

f = F[2]
fneu = Fneu[2]

def single_spine_peak_plotting(input_f, input_fneu):
    i=0
    corrected = input_f - (0.7*input_fneu)
        # a standard subtraction of fluorescence background immediately surrounding each ROI
    corrected_sample_trace = BaselineRemoval(corrected)
    corrected_trace = corrected_sample_trace.ZhangFit(repitition=100)
        # an adaptively weighted iterated modified polynomial fit that ignores peaks and corrects baseline to 0
    peaks, _ = find_peaks(corrected_trace, height = 3*(abs(np.median(corrected_trace)) + abs(corrected_trace.min())), distance = 20)
    amplitudes = corrected_trace[peaks] - np.median(corrected_trace)

    negative_points = np.where((corrected_trace < np.median(corrected_trace)))[0]
# print(negative_points)
    decay_points = []
    decay_time = []

    for i in peaks:
        negative_after_peak = negative_points[negative_points>i]
        if len(negative_after_peak > 0):
            decay_points.append(negative_after_peak[0])

    for k,a in zip(peaks, decay_points):
        decay_time.append(np.abs(a-k)/20)

        # scipy find_peaks function
        #then plot the traces you generate
    plt.plot(corrected_trace)
    plt.plot(peaks, corrected_trace[peaks], "x")
    plt.plot(np.full_like(corrected_trace, 3*(abs(np.median(corrected_trace)) + abs(corrected_trace.min()))), "--",color = "grey")
    plt.plot(np.full_like(corrected_trace, np.median(corrected_trace)), "--", color = 'r')
    plt.plot(np.full_like(decay_points, corrected_trace[decay_points], "d"))
    print('the peak time stamps are :{}' .format(peaks))
    print('the peak returns to baseline at frame: {}' .format(decay_points))
    print('the decay time in seconds for each peak is: {}' .format(decay_time))
    print('the amplitude of each peak is: {}' .format(amplitudes))

    plt.show()

def single_synapse_baseline_correction_and_peak_return(input_f, input_fneu, return_peaks = False, return_decay_times = False, return_amplitudes = False):
    
    corrected_trace = input_f - (0.7*input_fneu)
    corrected_trace = BaselineRemoval(corrected_trace)
    corrected_trace = corrected_trace.ZhangFit(repitition = 100)
    peaks, _ = find_peaks(corrected_trace, height = 3*(abs(np.median(corrected_trace)) + abs(corrected_trace.min())), distance = 40)
    amplitudes = corrected_trace[peaks] - np.median(corrected_trace)
    negative_points = np.where((corrected_trace < np.median(corrected_trace)))[0]
    decay_points = []
    decay_time = []

    for i in peaks:
        negative_after_peak = negative_points[negative_points>i]
        if len(negative_after_peak > 0):
            decay_points.append(negative_after_peak[0])

    next_peak_index = []
    for k,a in zip(peaks, decay_points):
        if next_peak_index < a:
            decay_time.append(np.nan)
        
        else:
            decay_time.append(np.abs(a-k)/20)
        
        
    if return_peaks == True:
        return peaks
    if return_decay_times == True:
        return decay_points
    if return_amplitudes == True:
        return amplitudes


In [ ]:
peak_test = []
amp_test = []
decay_test = []
n=0
for i,j in zip(F, Fneu):
    # peak_test.append(single_synapse_baseline_correction_and_peak_return(i,j, return_peaks=True))
    # amp_test.append(single_synapse_baseline_correction_and_peak_return(i,j, return_peaks=False, return_amplitudes=True))
    # decay_test.append(single_synapse_baseline_correction_and_peak_return(i,j, return_peaks=False, return_decay_times=True))
    print(n)
    n+=1
    single_spine_peak_plotting(i,j)

In [ ]:
print(decay_test)

In [ ]:
print(peak_test)

In [ ]:
decayed_library = []
for f, fneu in zip(F,Fneu):
    decayed_library.append(single_synapse_baseline_correction_and_peak_return(f, fneu, return_peaks = False, return_decay_times = True))


In [ ]:
desired_key = "lam"
low_std = []
for element in stat:
    if desired_key in element:
        low_std.append(element[desired_key])

np.mean(stat[601]['lam'])

In [ ]:
decayed_library
second_times = []
for i in decayed_library:
    second_times.append(i/10)
print(second_times) 


In [ ]:
#gives out the time course of each peak (currently threshold is set to 1.5x the noise)

f = F[2]
fneu = Fneu[2]
sample_trace = f - 0.7*fneu
corrected_sample_trace = BaselineRemoval(sample_trace)
corrected_trace = corrected_sample_trace.ZhangFit(lambda_=500)
threshold = 1.5*(abs(np.median(corrected_trace)) + abs(corrected_trace.min()))
baseline = np.median(corrected_trace)
# print(threshold)

peaks, _ = find_peaks(corrected_trace, height = 3*(abs(np.median(corrected_trace)) + abs(corrected_trace.min())), distance = 20)
negative_points = np.where((corrected_trace < np.median(corrected_trace)))[0]
# print(negative_points)
decay_points = []
decay_time = []

for i in peaks:
    negative_after_peak = negative_points[negative_points>i]
    if len(negative_after_peak > 0):
        decay_points.append(negative_after_peak[0])

next_peak_index =  []
for k,a in zip(peaks, decay_points):
    if len(peaks) > 1:
        next_peak_index = peaks[peaks > k].min()
        if next_peak_index < a:
            decay_time.append(np.nan)
        else:
            decay_time.append(np.abs(a-k) / 20)
    else:
        decay_time.append(np.abs(a-k)/20)

print(peaks)  
print('the peak returns to baseline at frame: {}' .format(decay_points))
print('the decay time for each peak is: {}' .format(decay_time))
# for i in peaks:
#     potential_time_points = np.where((corrected_trace < baseline))
#     indices = np.where(potential_time_points > peaks)
#     # if len(indices) > 0:
#         # peak_index = np.abs(indices - i).argmin()
#         # peak_indexes.append(indices[peak_index])
#     print("Peak index:", i, "Index where value equals threshold:", indices[peak_index])

len(decay_points) == len(peaks)


"""Write a code that goes through each of the peaks and decay times; as we add the decay frames to the peak, if there is a second / third peak before the decay time; we change the decay to NaN"""


In [ ]:

index = []
for x, y in zip(peaks, decay_points[:]):
    if x[0] < y[1]:
        print(True)
    else:
        break            


In [ ]:

decay_time = []
next_peak =  []
start = 0
for k,a in zip(peaks, decay_points):
    if len(peaks) > 1:
        next_peak = peaks[start + 1]
        if next_peak < a:
            decay_time.append(np.nan)
        else:
            decay_time.append(np.abs(a-k) / 20)
    else:
        decay_time.append(np.abs(a-k)/20)
    start+=1

In [ ]:
#final plotting of the output for testing
plt.figure(figsize=(36,6))
plt.plot(corrected_trace)
plt.plot(np.full_like(corrected_trace, threshold))
plt.plot(peaks, corrected_trace[peaks], "x")
plt.plot(np.full_like(corrected_trace, 3*(abs(np.median(corrected_trace)) + abs(corrected_trace.min()))), "--",color = "grey")
plt.plot(np.full_like(corrected_trace, np.median(corrected_trace)), "-",color = "red")


In [ ]:
#In theory how you remove the values that we set to zero in the initial pass (should 
#directly lead to a larger std dev of the data)

def first_pass_peak_removal():
    first_pass = np.copy(corrected_trace)
    first_pass_peaks, _ = find_peaks(first_pass, height=0.15*first_pass.max(), distance=5)
    peak_regions = []
    for peak in first_pass_peaks:
        start = max(0, peak-frames)
        end = min(len(first_pass)-1, peak+frames+1)
        peak_regions += list(np.arange(start, end))
    first_pass[peak_regions] = 0
    threshold = 3*np.std(first_pass)
    first_pass = np.delete(first_pass, peak_regions) # remove values corresponding to peak_regions
    return threshold, first_pass

threshold, first_pass = first_pass_peak_removal()
synapse_peaks, _ = find_peaks(first_pass, height=threshold, distance=20)

In [ ]:
def load(fp):
    stat = np.load(f"{fp}\stat.npy", allow_pickle = True)
    ops = np.load(f"{fp}\ops.npy", allow_pickle = True).item()
    F = np.load(f"{fp}\F.npy", allow_pickle = True)
    Fneu = np.load(f"{fp}\Fneu.npy", allow_pickle = True)
    #iscell / spks also can be repeated in this way
    return stat, ops, F, Fneu

stat, ops, F, Fneu = load(r'E:\230510_homeostatic_plasticity\49%exp\TTX\wtRt_DIV20_AAV1_0.1ul_GCaMP_hSyn_48h_TTX_well001_r001\suite2p\plane0')

f = F[7]
fneu = Fneu[7]

#Sandbox removal of peak regions and re-thresholding
def synapse_freq_and_amp_extraction(f, fneu, frames):
    """Here we define our primary extraction function for suite2p F / Fneu.npy files; these files are unpacked iteratively 
    with the statement 'for f, fneu in zip(F, Fneu): followed by the outer function; this will unpack the parent fluorescence 
    files and gather freq / amplitude measures"""
    fluoro_trace = f - 0.7*fneu
    corrected_sample_trace = BaselineRemoval(fluoro_trace)
    corrected_trace = corrected_sample_trace.ZhangFit()
    def first_pass_peak_removal():
        first_pass = np.copy(corrected_trace)
        first_pass_peaks, _ = find_peaks(first_pass, height = 1.5*np.std(first_pass), distance = 5)
        peak_regions = []
        for peak in first_pass_peaks:
            start = max(0, peak-frames)
            end = min(len(first_pass)-1, peak+frames+1)
        peak_regions += list(np.arange(start, end))
        # for peak in first_pass_peaks:
        #     peak_regions += list(np.arange(peak-frames, peak+frames+1))
        first_pass[peak_regions]=0
        threshold = 2.5*np.std(first_pass)
        # plt.plot(corrected_trace)
        return threshold
    threshold = first_pass_peak_removal()
    synapse_peaks, _ = find_peaks(corrected_trace, height = threshold, distance = 20)
    plt.ylim(-10,400)
    plt.plot(corrected_trace)
    plt.plot(synapse_peaks, corrected_trace[synapse_peaks], "x")
    plt.plot(np.full_like(corrected_trace, threshold), "--",color = "grey")
    plt.plot(np.full_like(corrected_trace, np.median(corrected_trace)), "--", color = 'r')
    plt.show()
    
for j, k in zip(F, Fneu):
    synapse_freq_and_amp_extraction(j, k, 10)

In [ ]:
for peak in first_pass_peaks:
    start = max(0, peak-frames)
    end = min(len(first_pass)-1, peak+frames+1)
    peak_regions += list(np.arange(start, end))

In [ ]:
for j, k in zip(F, Fneu):
    synapse_freq_and_amp_extraction(j, k, 40)

In [ ]:
fluoro_trace = f - 0.7*fneu
corrected_sample_trace = BaselineRemoval(fluoro_trace)
corrected_trace = corrected_sample_trace.ZhangFit()
plt.plot(corrected_trace)
plt.plot(np.full_like(corrected_trace, 10*np.std(trace)), '--', color = 'r')
plt.plot(np.full_like(corrected_trace, (trace.max()-trace.min()+np.median(corrected_trace))), '-', color = 'grey')
plt.plot(np.full_like(corrected_trace, (3*np.std(noise) + np.median(trace))), color = 'blue')

In [ ]:
correction = BaselineRemoval(fneu)
noise = correction.ZhangFit(repitition=100)
3*np.std(noise) + np.median(trace)
plt.plot(trace)

In [ ]:
# plt.plot(corrected_trace)
plt.plot(trace)
# plt.plot(trace, trace[peaks], "x")
plt.plot(np.full_like(trace, 10*np.std(trace)), "--",color = "grey")
plt.plot(np.full_like(trace, np.median(trace)), "--", color = 'r')
print(np.std(trace))
# np.std(trace)

In [ ]:
"""Create a copy of a baseline corrected fluorescence trace and remove the peak regions (+/- the surrounding area) to compute baseline"""

n = 40 # number of integers on either side
indices = []
for peak in peak_list:
    indices += list(np.arange(peak-n, peak+n+1))
trace[indices] = 0
plt.plot(trace)


# zero = np.copy(trace)
# zero[peak_list] = 0
# plt.plot(zero)

In [ ]:
trace[indices] = 0
plt.plot(trace)

In [ ]:
"""Figure plotting"""

